## Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

In [2]:
data = pd.read_csv('checkins.dat', sep='|', skipinitialspace=True, skiprows=[1], skipfooter=2).dropna()[:100000]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
data.columns = data.columns.str.strip()

In [4]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
7,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [5]:
data.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [6]:
data.shape

(100000, 6)

In [7]:
X = data[['latitude', 'longitude']]
X = X.applymap(float)

In [8]:
np.array(X)

array([[  38.8951118,  -77.0363658],
       [  33.800745 ,  -84.41052  ],
       [  45.5234515, -122.6762071],
       ...,
       [  29.7628844,  -95.3830615],
       [  32.802955 ,  -96.769923 ],
       [  37.7749295, -122.4194155]])

In [9]:
clust = MeanShift(bandwidth=0.1).fit(X)
X['cluster'] = clust.predict(X)

In [10]:
X.head()

,latitude,longitude,cluster
1,38.895112,-77.036366,5
3,33.800745,-84.410520,7
7,45.523452,-122.676207,30
9,40.764462,-111.904565,65
10,33.448377,-112.074037,1


In [11]:
cluster_size = pd.DataFrame(X.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['clust_size']

In [12]:
cluster_size.head()

,clust_size
cluster,
0,12506
1,4692
2,3994
3,3363
4,3526


In [13]:
cluster_centers_df = pd.DataFrame(clust.cluster_centers_)
cluster_centers_df.columns = ['cent_latitude', 'cent_longitude']

In [14]:
cluster_df = cluster_centers_df.join(cluster_size)
cluster_df.to_csv('clusters.csv', index = None)
cluster_df = cluster_df[cluster_df.clust_size > 15]
cluster_df.head()

,cent_latitude,cent_longitude,clust_size
0,40.717716,-73.991835,12506
1,33.449438,-112.002140,4692
2,33.446380,-111.901888,3994
3,41.878244,-87.629843,3363
4,37.688682,-122.409330,3526


In [15]:
office_coordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)]

In [16]:
def get_distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5


def get_min_distance_to_office(lat, lon):
    min_dist = None
    for (of_lat, of_lon) in office_coordinates:
        dist = get_distance(lat, lon, of_lat, of_lon)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist
    

In [17]:
cluster_df['min_distance'] = list(map(get_min_distance_to_office, cluster_df.cent_latitude, cluster_df.cent_longitude))

In [18]:
cluster_df.sort_values(by='min_distance').head()

,cent_latitude,cent_longitude,clust_size,min_distance
420,-33.860630,151.204776,28,0.007835
370,52.372964,4.892317,31,0.009353
419,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848
